In [1]:
import pandas as pd
import numpy as np

import seaborn as sns


In [4]:
train = pd.read_csv('./Assets/train_users_2.csv')
test = pd.read_csv('./Assets/test_users.csv')

In [5]:
print train.shape
print test.shape

(213451, 16)
(62096, 15)


In [4]:
brow_cnt = zip(train['first_browser'].value_counts().index, train['first_browser'].value_counts().values)
brow_other = [x[0] for x in brow_cnt if x[1]<150]
train.first_browser = train.first_browser.apply(lambda x: 'Other' if x in brow_other else x)

brow_cnt = zip(test['first_browser'].value_counts().index, test['first_browser'].value_counts().values)
brow_other = [x[0] for x in brow_cnt if x[1]<150]
test.first_browser = test.first_browser.apply(lambda x: 'Other' if x in brow_other else x)

In [5]:
features = ['gender', 'signup_method', 'language', 'signup_app', 'first_device_type','affiliate_provider', 'first_browser']
for fea in features:
    tr_fea = set(train[fea].unique())
    ts_fea = set(test[fea].unique())
    print fea,  tr_fea.symmetric_difference(ts_fea)

gender set([])
signup_method set(['weibo'])
language set(['hr', 'is', '-unknown-'])
signup_app set([])
first_device_type set([])
affiliate_provider set(['wayn'])
first_browser set(['Opera', 'AOL Explorer'])


In [6]:
test.drop(39024,inplace=True)

weibo = test[test.signup_method == 'weibo'].index
test.drop(weibo,inplace=True)

In [7]:
lang_is = train[train.language == 'is'].index
train.drop(lang_is,inplace=True)

lang_hr = train[train.language == 'hr'].index
train.drop(lang_hr,inplace=True)

wayn = train[train.affiliate_provider == 'wayn'].index
train.drop(wayn,inplace=True)

In [8]:
features = ['gender', 'signup_method', 'language', 'signup_app', 'first_device_type','affiliate_provider', 'first_browser']
for fea in features:
    tr_fea = set(train[fea].unique())
    ts_fea = set(test[fea].unique())
    print fea,  tr_fea.symmetric_difference(ts_fea)


gender set([])
signup_method set([])
language set([])
signup_app set([])
first_device_type set([])
affiliate_provider set([])
first_browser set(['Opera', 'AOL Explorer'])


### Equalize features in dataframes to be combined

In [9]:
[i for i in train.columns.values if i not in test.columns.values]

['country_destination']

In [10]:
# y_train = train[[i for i in train.columns.values if i not in test.columns.values]]
# train.drop([i for i in train.columns.values if i not in test.columns.values], inplace=True, axis = 1)
ids = train[['id', 'country_destination']]

In [11]:
combined = train.append(test, ignore_index=True)
combined.shape

(275508, 16)

In [ ]:
### age_x_test and age_x_train represent the set to give me ages
### with ids they will be merged later back into train and test originals

-------------------

In [12]:
age_not_null = combined[combined.age.notnull()] 

In [18]:
age_not_null.shape

(158665, 16)

In [13]:
y_age_target = age_not_null.age
y_age_target = y_age_target.apply(lambda x: x if x < 75 else 75).apply(lambda x: x if x > 18 else 18)
y_age_target.unique()

array([ 38.,  56.,  42.,  41.,  46.,  47.,  50.,  36.,  37.,  33.,  31.,
        29.,  30.,  40.,  26.,  32.,  35.,  59.,  49.,  44.,  34.,  28.,
        19.,  53.,  52.,  39.,  57.,  25.,  54.,  69.,  63.,  43.,  55.,
        65.,  58.,  61.,  18.,  27.,  45.,  60.,  48.,  51.,  64.,  72.,
        70.,  67.,  73.,  75.,  66.,  68.,  24.,  62.,  23.,  74.,  71.,
        22.,  21.,  20.])

In [14]:
age_not_null_f = age_not_null[features]
age_not_null_f_dum = pd.get_dummies(age_not_null_f)
age_not_null_f_dum.shape

(158665, 71)

-------------------

In [20]:
age_null = combined[combined.age.isnull()]

In [21]:
age_null.age.unique()

array([ nan])

In [22]:
print age_null.shape
age_null.head()

(116843, 16)


,affiliate_channel,affiliate_provider,age,country_destination,date_account_created,date_first_booking,first_affiliate_tracked,first_browser,first_device_type,gender,id,language,signup_app,signup_flow,signup_method,timestamp_first_active
0,direct,direct,NaN,NDF,2010-06-28,NaN,untracked,Chrome,Mac Desktop,-unknown-,gxn3p5htnn,en,Web,0,facebook,20090319043255
5,other,other,NaN,US,2010-01-01,2010-01-02,omg,Chrome,Mac Desktop,-unknown-,osr2jwljor,en,Web,0,basic,20100101215619
12,direct,direct,NaN,FR,2010-01-05,2010-01-18,NaN,-unknown-,Other/Unknown,-unknown-,k6np330cm1,en,Web,0,basic,20100105060859
16,other,craigslist,NaN,NDF,2010-01-07,NaN,NaN,-unknown-,Other/Unknown,-unknown-,2dwbwkx056,en,Web,0,basic,20100107215125
18,seo,facebook,NaN,NDF,2010-01-08,NaN,NaN,-unknown-,Other/Unknown,-unknown-,cxlg85pg1r,en,Web,0,basic,20100108015641


In [23]:
age_null_f = age_null[features]
age_null_f_dum = pd.get_dummies(age_null_f)
age_null_f_dum.shape

(116843, 71)

In [24]:
[i for i in age_not_null_f_dum.columns.values if i not in age_not_null_f_dum.columns.values]

[]

-------------------

--------------------------------------
## Imputation

In [25]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler


In [26]:
X_train, X_test, y_train, y_test = train_test_split(age_not_null_f_dum, y_age_target, test_size=0.33, random_state=42)

In [27]:
print len(y_test)

52360


In [28]:
len(y_age_target)

158665

### Evaluation

In [29]:
def evaluate_model(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    a = accuracy_score(y_test, y_pred)
#     cm = confusion_matrix(y_test, y_pred)
#     cr = classification_report(y_test, y_pred)
    return a

In [30]:
# sns.heatmap(acc_eval.T,square=True,annot=True)
# plt.xlabel('true label')
# plt.ylabel('predicted label')

### Prediction 1

In [31]:
model = RandomForestClassifier()
model.fit(age_not_null_f_dum, y_age_target)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [32]:
y_pred = model.predict(age_null_f_dum)
np.unique(y_pred)

array([ 18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,
        29.,  30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.,
        40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.,  50.,
        51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.,  61.,  62.,
        63.,  64.,  65.,  66.,  67.,  68.,  69.,  70.,  71.,  72.,  74.,
        75.])

In [33]:
y_pred.shape

(116843L,)

In [34]:
evaluate_model(model)

0.047536287242169593

### Cross Validation

In [35]:
params = {'n_estimators':[3, 5, 10, 50],
          'criterion': ['gini', 'entropy'],
          'max_depth': [None, 3, 5],
          'min_samples_split': [2,5],
          'class_weight':[None, 'balanced']}
gs_rf = GridSearchCV(RandomForestClassifier(), params, cv=KFold(len(y_age_target), n_folds=3, shuffle=True))

In [36]:
gs_rf.fit(age_not_null_f_dum, y_age_target)

KeyboardInterrupt: 

In [37]:
print gs_rf.best_params_
print gs_rf.best_score_
print gs_rf.best_estimator_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

### Prediction 2

In [38]:
model_rf = RandomForestClassifier(min_samples_split=2, n_estimators=10,
                                  criterion='entropy', max_depth=5, class_weight=None)
model_rf.fit_transform(age_not_null_f_dum, y_age_target)

C:\Users\ali\Anaconda2\lib\site-packages\sklearn\utils\deprecation.py:70: DeprecationWarning: Function transform is deprecated; Support to use estimators as feature selectors will be removed in version 0.19. Use SelectFromModel instead.
  warnings.warn(msg, category=DeprecationWarning)


array([[ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  1.,  0., ...,  0.,  1.,  0.],
       [ 0.,  1.,  0., ...,  1.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  0.,  1.,  0.],
       [ 0.,  1.,  0., ...,  0.,  0.,  0.]])

In [39]:
rf_prdeict = model_rf.predict(age_null_f_dum)
np.unique(rf_prdeict)

array([ 18.,  21.,  22.,  23.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,
        32.,  33.,  34.,  35.,  36.,  37.,  53.,  55.,  75.])

In [40]:
print rf_prdeict.shape
rf_prdeict

(116843L,)


array([ 28.,  28.,  30., ...,  30.,  32.,  29.])

In [41]:
evaluate_model(model_rf)

0.049025974025974028

---------------------
## Filling in null values in age column of train dataset

In [42]:
print len(train)
print len(train[train.age.isnull() == True]), '+',len(test[test.age.isnull() == True]), '=', (
    len(train[train.age.isnull() == True])+len(test[test.age.isnull() == True]))
print len(age_null)
print len(rf_prdeict)

213436
87984 + 28859 = 116843
116843
116843


In [ ]:
# age = pd.Series(rf_prdeict,index=age_null.index,name='age')
# age_id_df = age_null['id']
# age_id_df = pd.concat([age_id_df, age], axis=1)
# age_id_df

In [ ]:
# train_final[train_final.age_x.isnull()]
# pd.merge(left=train, right=age_id_df, on='id',how='inner')

In [40]:
# b = age_null.drop('age',axis=1)
# age = pd.Series(rf_prdeict,index=age_null.index,name='age')
# b = pd.concat([b,age],axis=1)
# c = age_null.copy()#['age'] = 
# c['age'] = 

In [43]:
age_null['age'] = pd.Series(rf_prdeict,index=age_null.index,name='age')
print age_null.age.shape
age_null.age.unique()

(116843L,)


C:\Users\ali\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


array([ 28.,  30.,  29.,  32.,  33.,  25.,  75.,  31.,  26.,  27.,  53.,
        36.,  22.,  35.,  34.,  18.,  21.,  23.,  37.,  55.])

In [44]:
a_train = pd.merge( left=train, right= age_null , on= 'id', how = 'inner' )
a1_train = pd.merge( left=train, right=age_not_null, on= 'id', how = 'inner' )

In [45]:
train_merge = pd.concat([a_train, a1_train], axis=0)
train_merge.columns

Index([u'id', u'date_account_created_x', u'timestamp_first_active_x',
       u'date_first_booking_x', u'gender_x', u'age_x', u'signup_method_x',
       u'signup_flow_x', u'language_x', u'affiliate_channel_x',
       u'affiliate_provider_x', u'first_affiliate_tracked_x', u'signup_app_x',
       u'first_device_type_x', u'first_browser_x', u'country_destination_x',
       u'affiliate_channel_y', u'affiliate_provider_y', u'age_y',
       u'country_destination_y', u'date_account_created_y',
       u'date_first_booking_y', u'first_affiliate_tracked_y',
       u'first_browser_y', u'first_device_type_y', u'gender_y', u'language_y',
       u'signup_app_y', u'signup_flow_y', u'signup_method_y',
       u'timestamp_first_active_y'],
      dtype='object')

In [46]:
train_merge.drop([i for i in train_merge.columns if '_x' in i], axis=1, inplace = True)
train_merge.columns = train_merge.columns.str.replace('_y', '')
train_merge.columns

Index([u'id', u'affiliate_channel', u'affiliate_provider', u'age',
       u'country_destination', u'date_account_created', u'date_first_booking',
       u'first_affiliate_tracked', u'first_browser', u'first_device_type',
       u'gender', u'language', u'signup_app', u'signup_flow', u'signup_method',
       u'timestamp_first_active'],
      dtype='object')

In [47]:
train_merge.age.isnull().sum()

0

In [48]:
print train.shape 
print train_merge.shape

(213436, 16)
(213436, 16)


In [49]:
train_merge.reset_index(drop=True,inplace=True)

In [50]:
train_merge.to_pickle('train_age')